In [5]:
import numpy as np 
import pandas as pd 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/pan-iit/training"))

# Any results you write to the current directory are saved as output.

['solution.csv', 'training']


In [ ]:
path = '../input/pan-iit/training/solution.csv'
data = pd.read_csv(path)
data.head()

In [2]:
import cv2
from tqdm import tqdm

In [ ]:
train_data = []
labels = []
train_path = '../input/pan-iit/training/training'
for file in tqdm(os.listdir(train_path)):
    path = os.path.join(train_path, file)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (200, 200))
    train_data.append(np.array(img))
    labels.append(data['category'][int(file.split('.')[0])-1])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
model = Sequential()
model.add(Convolution2D(32,3,data_format='channels_last',activation='relu',input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])

In [ ]:
X = train_data
X = np.array(X).reshape(len(X),200,200,1)
X = X/255.0
y = np.array(to_categorical(labels))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 16)

In [ ]:
classifier.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
import keras
import keras.backend as K 
from keras.models import Model
from keras.layers import Dense, Dropout, Add, Input, BatchNormalization, Activation
from keras.layers import  Conv2D, MaxPooling2D, AveragePooling2D, Flatten
from keras.regularizers import l2

def main_block(x, filters, n, strides, dropout):
    # Normal part
    x_res = Conv2D(filters, (3,3), strides=strides, padding="same")(x)# , kernel_regularizer=l2(5e-4)
    x_res = BatchNormalization()(x_res)
    x_res = Activation('relu')(x_res)
    x_res = Conv2D(filters, (3,3), padding="same")(x_res)
    # Alternative branch
    x = Conv2D(filters, (1,1), strides=strides)(x)
    # Merge Branches
    x = Add()([x_res, x])

    for i in range(n-1):
        # Residual conection
        x_res = BatchNormalization()(x)
        x_res = Activation('relu')(x_res)
        x_res = Conv2D(filters, (3,3), padding="same")(x_res)
        # Apply dropout if given
        if dropout: x_res = Dropout(dropout)(x)
        # Second part
        x_res = BatchNormalization()(x_res)
        x_res = Activation('relu')(x_res)
        x_res = Conv2D(filters, (3,3), padding="same")(x_res)
        # Merge branches
        x = Add()([x, x_res])

    # Inter block part
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def build_model(input_dims, output_dim, n, k, act= "relu", dropout=None):
    """ Builds the model. Params:
            - n: number of layers. WRNs are of the form WRN-N-K
                 It must satisfy that (N-4)%6 = 0
            - k: Widening factor. WRNs are of the form WRN-N-K
                 It must satisfy that K%2 = 0
            - input_dims: input dimensions for the model
            - output_dim: output dimensions for the model
            - dropout: dropout rate - default=0 (not recomended >0.3)
            - act: activation function - default=relu. Build your custom
                   one with keras.backend (ex: swish, e-swish)
    """
    # Ensure n & k are correct
    assert (n-4)%6 == 0
    assert k%2 == 0
    n = (n-4)//6 
    # This returns a tensor input to the model
    inputs = Input(shape=(input_dims))

    # Head of the model
    x = Conv2D(16, (3,3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 3 Blocks (normal-residual)
    x = main_block(x, 16*k, n, (1,1), dropout) # 0
    x = main_block(x, 32*k, n, (2,2), dropout) # 1
    x = main_block(x, 64*k, n, (2,2), dropout) # 2
            
    # Final part of the model
    x = AveragePooling2D((8,8))(x)
    x = Flatten()(x)
    outputs = Dense(output_dim, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


if __name__ == "__main__":
    model = build_model((100,100,1), 6, 22, 8)
    model.summary()

In [ ]:
shape, classes = (200, 200, 1), 7
model = build_model(shape, classes, 22, 4)
model.compile("adam", "categorical_crossentropy", ["accuracy"])
model.fit(X, y, epochs=40)

In [3]:
from keras.models import model_from_json

Using TensorFlow backend.


In [7]:
print(os.listdir("../input/kernel6b8f93f8da"))

['model_300.json', 'model_300.h5', '__output__.json', 'custom.css', '__results__.html']


In [8]:
model_json = model.to_json()
with open("../input/kernel6b8f93f8da/model_300.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../input/kernel6b8f93f8da/model_300.h5")
print("Saved model to disk")

NameError: name 'model' is not defined

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
import os

In [4]:
test_data = []
sno = []
test_path = 'PanIIT/testing'
for file in tqdm(os.listdir(test_path)):
    path = os.path.join(test_path, file)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (225, 225))
    name = int(file.split('.')[0])
    sno.append(name)
    test_data.append(np.array(img))

100%|██████████| 40000/40000 [02:09<00:00, 341.29it/s]


In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json

Using TensorFlow backend.


In [ ]:
test_data

In [14]:
X1 = test_data[20001:]
X1 = np.array(X1).reshape(len(X1),225,225,1)
X1 = X1/255.0

In [ ]:
X2 = test_data[20000:30000]
X2 = np.array(X2).reshape(len(X2),225,225,1)
X2 = X2/255.0

In [ ]:
X3 = test_data[30000:40000]
X3= np.array(X3).reshape(len(X3),225,225,1)
X3 = X3/255.0

In [ ]:
print(os.listdir("../input/"))

In [7]:
# load json and create model
json_file = open('../input/kernel6b8f93f8da/model_300.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../input/kernel6b8f93f8da/model_300.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])


Loaded model from disk


In [8]:
label = []

In [15]:
for i in tqdm(range(len(X1))):
    y = loaded_model.predict(X1[i:i+1])
    label.append(np.argmax(y))

100%|██████████| 19999/19999 [12:38<00:00, 26.36it/s]


In [25]:
y = loaded_model.predict(test_data[20000:20001])
print(np.argmax(y))
print(sno[20000])

ValueError: attempt to get argmax of an empty sequence

In [19]:
l = pd.DataFrame(data = label)

In [20]:
s = pd.DataFrame(data = sno)

In [10]:
data = { 'id': sno[:20000], 'category': label}
submission5 = pd.DataFrame(data)
submission5.head()

,id,category
0,21942,5
1,17797,1
2,11444,6
3,2405,5
4,32869,6


In [ ]:
y_0 = loaded_model.predict(X[:0])

In [ ]:
for i in tqdm(range(len(X))):
    y = loaded_model.predict(X[i:i+1])
    label.append(np.argmax(y))

In [ ]:
yolo = [1,2,2,3,40]
yolo[0:3]

In [11]:
# submission1.to_csv('1.csv',index=False)
# submission2.to_csv('2.csv',index=False)
submission5.to_csv('5.csv',index=False)

In [22]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "sno.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


In [23]:
create_download_link(s)

In [ ]:
label